In [1]:
import requests
import datetime
from urllib.parse import urlencode
import json

In [2]:
import base64

In [3]:
client_ID = '3e0b06af25054282a033710bb1736c42'
client_secret = '37dc1dbb503242f59e9734d02f3067dc'

In [4]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist', limit=20 ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [5]:
spotify = SpotifyAPI(client_ID, client_secret)

In [6]:
a= spotify.search({"track": "perreo"}, search_type="track")

q=track%3Aperreo&type=track


In [8]:
a['tracks']

{'href': 'https://api.spotify.com/v1/search?query=track%3Aperreo&type=track&offset=0&limit=20',
 'items': [{'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4q3ewBCX7sLwd24euuV69X'},
      'href': 'https://api.spotify.com/v1/artists/4q3ewBCX7sLwd24euuV69X',
      'id': '4q3ewBCX7sLwd24euuV69X',
      'name': 'Bad Bunny',
      'type': 'artist',
      'uri': 'spotify:artist:4q3ewBCX7sLwd24euuV69X'}],
    'available_markets': ['AD',
     'AE',
     'AL',
     'AR',
     'AT',
     'AU',
     'BA',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'BY',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
     'GR',
     'GT',
     'HK',
     'HN',
     'HR',
     'HU',
     'ID',
     'IE',
     'IL',
     'IN',
     'IS',
     'IT',
     'JO',
     'JP',
     'KW',
     'K

In [25]:
l = spotify.search({"track": "perreo"}, search_type="track")


q=track%3Aperreo&type=track


In [26]:
for i in l['tracks']['items']:
    print(len(i['album']['artists']))
    for index,item in enumerate(i['album']['artists']):
        print(json.dumps(i['album']['artists'][index]['name'], indent=4))
    print()
    print()

1
"Bad Bunny"


3
"Ankhal"
"Farruko"
"Guaynaa"


3
"Rich Music LTD"
"Sech"
"Dalex"


3
"Cauty"
"Noriel"
"Jowell & Randy"


2
"Jossex Rosy"
"Charlie Axel"


1
"Nobeat"


1
"Feid"


1
"Luciano DJ"


2
"Roa los Santos"
"Mister Remix"


1
"Uzielito Mix"


2
"Roa los Santos"
"Dj Free"


2
"Roa los Santos"
"Dj Free"


1
"Roa los Santos"


2
"Roa los Santos"
"Mister Remix"


1
"Yomille Omar"


1
"Jexal"


1
"Jexal"


1
"Jexal"


3
"Ele A El Dominio"
"Jon Z"
"Papi Emo"


1
"DJ Kelvin El Sacamostro"




In [35]:
for i in l['tracks']['items']:
    print(i['popularity'], i['duration_ms'], i['explicit'], i['name'])
#     for index,item in enumerate(i['album']['artists']):
#         print(json.dumps(i['album']['artists'][index], indent=4))
#     print(json.dumps(i,indent=4))
    print()

90 172127 False Yo Perreo Sola

72 239586 True Perreo Intenso

76 218586 True Perreo en La Luna

54 338527 False El Perreo Intenso

44 155062 True Perreo Es Lo Que ES

51 213355 False Perreo Sano

54 91653 False PERREOXOXO

62 224862 False Intro Teléfono Perreo Rkt - feat. Mister Remix

40 152840 False Perreo Bendecido

60 205408 True Suena Perreo

37 143720 True Perreo Bendecido 2

38 176866 True Yo Lo Que Quiero Es Perreo - Freestyle Version

37 131973 True Yo Lo Que Quiero es Perreo

37 137442 True Perreo Bendecido - Versión Instrumental

31 202840 True Perreo Asqueroso

34 153810 True Chingo De Perreo

34 155063 True Chingo De Perreo Mix

34 172774 True Chindo De Perreo Mix

49 197647 True Perreo HP - Remix

54 272677 True Candy Perreo



In [38]:
f = []
for index,item in enumerate(l['tracks']['items']):
    d = {}
    d['Id'] = index
    d['item'] ={
        'songName' : item['popularity']
    }
    f.append(d)
#     print(i['popularity'], i['duration_ms'], i['explicit'], i['name'], i['album']['artists'][index]['name'])

In [39]:
f

[{'Id': 0, 'item': {'songName': 90}},
 {'Id': 1, 'item': {'songName': 72}},
 {'Id': 2, 'item': {'songName': 76}},
 {'Id': 3, 'item': {'songName': 54}},
 {'Id': 4, 'item': {'songName': 44}},
 {'Id': 5, 'item': {'songName': 51}},
 {'Id': 6, 'item': {'songName': 54}},
 {'Id': 7, 'item': {'songName': 62}},
 {'Id': 8, 'item': {'songName': 40}},
 {'Id': 9, 'item': {'songName': 60}},
 {'Id': 10, 'item': {'songName': 37}},
 {'Id': 11, 'item': {'songName': 38}},
 {'Id': 12, 'item': {'songName': 37}},
 {'Id': 13, 'item': {'songName': 37}},
 {'Id': 14, 'item': {'songName': 31}},
 {'Id': 15, 'item': {'songName': 34}},
 {'Id': 16, 'item': {'songName': 34}},
 {'Id': 17, 'item': {'songName': 34}},
 {'Id': 18, 'item': {'songName': 49}},
 {'Id': 19, 'item': {'songName': 54}}]

In [43]:
with open('./test.json', 'w') as file:
    json.dump(f , file,sort_keys=True,indent=4)